<a href="https://colab.research.google.com/github/Grajes-pl/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import  to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [2]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_3/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_3/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/train.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
# if aby ta kategoryzacja nie wykonała się kilka razy jakby ktoś tę komórkę odpalił wieceij niż raz

if y_train.ndim ==1: y_train = to_categorical(y_train)
if y_test.ndim ==1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(get_model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(
      X_train, 
      y_train,
      batch_size= params_fit.get('batch_size', 128),
      epochs= params_fit.get('epochs', 5),
      verbose= params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]
  )
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape ),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),  
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),  
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),  
])

In [0]:
def func_obj(params):
  model = get_model(params)

  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size= int(params.get('batch_size', 128)),
      epochs= 5,
      verbose= 0,

  )
  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status':STATUS_OK, 'model':model}

In [8]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    20,
    Trials()
)

Streaming output truncated to the last 5000 lines.
 - ETA: 2s - loss: 0.4492 - accuracy: 0.8702

 489/1088 [============>.................]
 - ETA: 2s - loss: 0.4424 - accuracy: 0.8719

 503/1088 [============>.................]
 - ETA: 2s - loss: 0.4358 - accuracy: 0.8736

 517/1088 [=============>................]
 - ETA: 2s - loss: 0.4380 - accuracy: 0.8729

 529/1088 [=============>................]
 - ETA: 2s - loss: 0.4395 - accuracy: 0.8731

 544/1088 [==============>...............]
 - ETA: 2s - loss: 0.4311 - accuracy: 0.8758
